In [1]:
import pandas as pd

In [2]:
path = "../data/final/"
DATA = pd.read_csv(path+"chart.csv")

In [3]:
DATA.head(5)

,sentiment_comp,sentiment_pos,sentiment_neg,sentiment_neu,f_k_grade,flesch_index,fog_index,difficult_words,num_syllables,num_words,num_lines,num_dup,artist,song,lyricId
0,0.3568,0.164,0.160,0.676,1.7,97.09,3.75,40,541,469,58,0,Marty Robbins,El Paso,1
1,0.9991,0.532,0.031,0.437,-0.4,107.79,2.62,3,175,154,26,4,Frankie Avalon,Why,2
2,0.9982,0.322,0.022,0.657,2.2,90.46,2.52,5,275,220,35,9,Johnny Preston,Running Bear,4
3,0.9924,0.280,0.119,0.601,1.2,103.83,3.92,3,243,216,22,9,Guy Mitchell,Heartaches By The Number,5
4,-0.9698,0.030,0.228,0.742,0.4,105.86,3.52,1,110,101,13,3,Paul Anka,It's Time To Cry,6


In [4]:
# need only lyrics
lyIdList = DATA["lyricId"]
lyric_path = "../data/interim/lyrics/lyric_"
text_data = []
for idx in lyIdList:
    l = ""
    with open(lyric_path+str(idx)+".txt", 'r') as lyricsFile:
        l = lyricsFile.read()
    text_data.append({
        "lyricId" : idx,
        "text" : l
    })
LYRICS = pd.DataFrame(text_data)

In [5]:
LYRICS.head(3)

,lyricId,text
0,1,Out in the West Texas town of El Paso\nI fell ...
1,2,I'll never let you go\nWhy? Because I love you...
2,4,"On the bank of the river\nStood Running Bear, ..."


In [6]:
# Load the regular expression library
import re

# Remove punctuation
LYRICS['text_processed'] = LYRICS['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
LYRICS['text_processed'] = LYRICS['text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
LYRICS['text_processed'].head()

0    out in the west texas town of el paso\ni fell ...
1    i'll never let you go\nwhy because i love you\...
2    on the bank of the river\nstood running bear y...
3    heartaches by the number troubles by the score...
4    when somebody leaves you that's the time to cr...
Name: text_processed, dtype: object

In [7]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = LYRICS.text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['out', 'in', 'the', 'west', 'texas', 'town', 'of', 'el', 'paso', 'fell', 'in', 'love', 'with', 'mexican', 'girl', 'nighttime', 'would', 'find', 'me', 'in', 'rosa', 'cantina', 'music', 'would', 'play', 'and', 'feleena', 'would', 'whirl', 'blacker']


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['chorus', 'verse'])

[nltk_data] Downloading package stopwords to /home/aps/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
!python -m spacy download en_core_web_sm

  Using cached en_core_web_sm-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:10])

['fall', 'love', 'mexican', 'girl', 'nighttime', 'would', 'find', 'music', 'would', 'play']


In [13]:

import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 3), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 1), (29, 2)]


In [42]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [43]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.149*"take" + 0.069*"light" + 0.067*"turn" + 0.064*"high" + 0.054*"world" '
  '+ 0.048*"fire" + 0.033*"burn" + 0.033*"fly" + 0.029*"touch" + 0.016*"hand"'),
 (1,
  '0.046*"go" + 0.035*"night" + 0.028*"day" + 0.026*"away" + 0.019*"life" + '
  '0.019*"time" + 0.017*"dream" + 0.015*"fall" + 0.014*"still" + '
  '0.013*"never"'),
 (2,
  '0.107*"go" + 0.105*"baby" + 0.104*"get" + 0.057*"want" + 0.045*"know" + '
  '0.021*"let" + 0.020*"keep" + 0.020*"need" + 0.019*"girl" + 0.017*"right"'),
 (3,
  '0.065*"say" + 0.032*"know" + 0.032*"tell" + 0.028*"look" + 0.026*"see" + '
  '0.026*"think" + 0.020*"could" + 0.018*"call" + 0.016*"way" + 0.015*"walk"'),
 (4,
  '0.173*"let" + 0.161*"come" + 0.077*"back" + 0.049*"feel" + 0.034*"song" + '
  '0.026*"bring" + 0.026*"sing" + 0.025*"rain" + 0.019*"music" + 0.019*"play"'),
 (5,
  '0.051*"get" + 0.025*"bitch" + 0.015*"know" + 0.014*"make" + 0.012*"go" + '
  '0.012*"money" + 0.011*"fuck" + 0.010*"shit" + 0.010*"niggas" + 0.010*"put"'),
 (6,
  '0.1

In [44]:
#Compute Model Perplexity and Coherence Score
#Let’s calculate the baseline coherence score
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.35953852426556737


In [53]:
# tuning for nuber of topics, alpha, beta
def compute_coherence_values(corpus, dictionary, k, a, b):
    #print(corpus[:1][0][:10])
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [54]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 3
max_topics = 30
step_size = 3
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../data/interim/lda_tuning_results.csv', index=False)
    pbar.close()





  0%|          | 0/540 [00:18<?, ?it/s]




  0%|          | 0/540 [02:44<?, ?it/s]




  0%|          | 0/540 [08:05<?, ?it/s]




  1%|          | 3/540 [07:47<22:54:04, 153.53s/it]



  1%|          | 4/540 [10:26<23:05:03, 155.04s/it]



  1%|          | 5/540 [13:04<23:11:36, 156.07s/it]



  1%|          | 6/540 [15:49<23:31:34, 158.60s/it]



  1%|▏         | 7/540 [18:32<23:42:24, 160.12s/it]



  1%|▏         | 8/540 [21:19<23:56:08, 161.97s/it]



  2%|▏         | 9/540 [24:08<24:14:07, 164.31s/it]



  2%|▏         | 10/540 [26:51<24:07:22, 163.85s/it]



  2%|▏         | 11/540 [29:38<24:12:13, 164.71s/it]



  2%|▏         | 12/540 [32:17<23:53:49, 162.93s/it]



  2%|▏         | 13/540 [34:57<23:44:10, 162.15s/it]



  3%|▎         | 14/540 [37:38<23:39:08, 161.88s/it]



  3%|▎         | 15/540 [40:16<23:25:09, 160.59s/it]



  3%|▎         | 16/540 [42:47<22:58:17, 157.82s/it]



  3%|▎         | 17/540 [45:18<22:38:08, 155.81s/it]



  3%|▎         | 18/540 [47:47<

 41%|████      | 222/540 [9:49:00<18:37:54, 210.93s/it]



 41%|████▏     | 223/540 [9:51:48<17:26:01, 197.99s/it]



 41%|████▏     | 224/540 [9:54:16<16:04:16, 183.09s/it]



 42%|████▏     | 225/540 [9:58:05<17:13:26, 196.84s/it]



 42%|████▏     | 226/540 [10:02:01<18:11:17, 208.53s/it]



 42%|████▏     | 227/540 [10:05:40<18:23:55, 211.62s/it]



 42%|████▏     | 228/540 [10:08:24<17:06:07, 197.33s/it]



 42%|████▏     | 229/540 [10:10:49<15:41:45, 181.69s/it]



 43%|████▎     | 230/540 [10:14:40<16:54:34, 196.37s/it]



 43%|████▎     | 231/540 [10:18:38<17:55:22, 208.81s/it]



 43%|████▎     | 232/540 [10:21:54<17:33:04, 205.14s/it]



 43%|████▎     | 233/540 [10:24:35<16:21:14, 191.77s/it]



 43%|████▎     | 234/540 [10:27:04<15:12:17, 178.88s/it]



 44%|████▎     | 235/540 [10:30:58<16:34:32, 195.65s/it]



 44%|████▎     | 236/540 [10:35:02<17:44:34, 210.11s/it]



 44%|████▍     | 237/540 [10:38:23<17:26:18, 207.19s/it]



 44%|████▍     | 238/540 [10:41:07<16:17:43,

 83%|████████▎ | 450/540 [22:51:46<5:43:27, 228.97s/it]



 84%|████████▎ | 451/540 [22:56:14<5:56:53, 240.60s/it]



 84%|████████▎ | 452/540 [23:00:15<5:53:01, 240.70s/it]



 84%|████████▍ | 453/540 [23:03:47<5:36:32, 232.10s/it]



 84%|████████▍ | 454/540 [23:07:04<5:17:40, 221.64s/it]



 84%|████████▍ | 455/540 [23:11:37<5:35:29, 236.82s/it]



 84%|████████▍ | 456/540 [23:16:14<5:48:29, 248.92s/it]



 85%|████████▍ | 457/540 [23:20:29<5:46:51, 250.74s/it]



 85%|████████▍ | 458/540 [23:24:10<5:30:30, 241.83s/it]



 85%|████████▌ | 459/540 [23:27:36<5:12:05, 231.18s/it]



 85%|████████▌ | 460/540 [23:32:04<5:23:08, 242.35s/it]



 85%|████████▌ | 461/540 [23:36:21<5:24:42, 246.61s/it]



 86%|████████▌ | 462/540 [23:40:32<5:22:09, 247.82s/it]



 86%|████████▌ | 463/540 [23:44:17<5:09:15, 240.98s/it]



 86%|████████▌ | 464/540 [23:47:46<4:53:22, 231.61s/it]



 86%|████████▌ | 465/540 [23:52:05<4:59:32, 239.63s/it]



 86%|████████▋ | 466/540 [23:56:24<5:02:49, 245.54s/it]


In [14]:
# from 2.1-hyper parameter search
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [15]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.420057388514231


In [18]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/home/aps/anaconda3/envs/songDA/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.424964  0.039063       1        1  49.323110
12    -0.381180  0.069064       2        1  24.910861
5     -0.293646  0.001738       3        1  15.375526
7     -0.130746 -0.236368       4        1   7.190223
2      0.095386  0.100168       5        1   1.364785
9      0.109351  0.017325       6        1   0.462540
11     0.114501 -0.007248       7        1   0.278391
4      0.112486  0.000595       8        1   0.250790
13     0.111826  0.004372       9        1   0.183824
10     0.115762  0.003398      10        1   0.166373
8      0.110462 -0.003496      11        1   0.161526
0      0.120161  0.007894      12        1   0.159757
1      0.112633 -0.000214      13        1   0.066579
3      0.112849  0.000814      14        1   0.053769
14     0.115119  0.002897      15        1   0.051946, topic_info=          Term          Freq         Total Category  logprob  loglift
236       baby  23453.000000  23453.000000  Default  30.0000  30.0000
59      little   7206.000000   7206.000000  Default  29.0000  29.0000
41         get  66349.000000  66349.000000  Default  28.0000  28.0000
62        love  52818.000000  52818.000000  Default  27.0000  27.0000
43          go  60981.000000  60981.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
34716    ruhum      1.332933      8.296913  Topic15  -6.9486   5.7342
34717    saate      1.332933      8.296913  Topic15  -6.9486   5.7342
34718  sanırım      1.332933      8.296913  Topic15  -6.9486   5.7342
34720  uzgunum      1.332933      8.296913  Topic15  -6.9486   5.7342
23202   whoomp      1.077216      7.114193  Topic15  -7.1616   5.6750

[773 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
366        1  0.958924  afraid
366        2  0.020752  afraid
366        3  0.013835  afraid
366        4  0.005188  afraid
366        5  0.000865  afraid
...      ...       ...     ...
33722      1  0.190953      잡아
33722      2  0.190953      잡아
33722      3  0.190953      잡아
33722      4  0.190953      잡아
33722      5  0.190953      잡아

[3717 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 13, 6, 8, 3, 10, 12, 5, 14, 11, 9, 1, 2, 4, 15])